# Ensemble Methods

## Quick Motivation

If one person is correct 60% of the time, how often is the majority of 99 similar people correct? (each is right 60% of the time in a binary classification task)

In [ ]:
from math import factorial, comb

## Voting 

Voting is a simple ensemble method. Multiple models are trained on the same data and each votes on the outcome of a classification take. The majority classification is the one chosen (this is called 'hard voting'). If the models can output a "confidence score" then these scores are averaged to produce a "soft" vote.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import numpy as np

In [ ]:
from sklearn.datasets import make_moons

We use the well-known 'moons' dataset with some noise

In [ ]:
X,y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [ ]:
# Plot the moons dataset
# Create the plot
plt.figure(figsize=(8, 6))
plt.scatter(x=X[y == 0, 0], y=X[y == 0, 1], color='blue', label='Class 0', alpha=0.6, edgecolor='k')
plt.scatter(x=X[y == 1, 0], y=X[y == 1, 1], color='red', label='Class 1', alpha=0.6, edgecolor='k')

# Enhance plot aesthetics
plt.title('Make Moons Dataset', fontsize=16)
plt.xlabel('Feature 1', fontsize=14)
plt.ylabel('Feature 2', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.show()

The `VotingClassifier` takes a list of voters as its input. It runs each submodel, trains it, then uses the voting method as the final output

In [ ]:
voting_clf = VotingClassifier(
    estimators=[
        ('logistic regression', LogisticRegression(random_state = 420)),
        ('Decision Tree', DecisionTreeClassifier(random_state = 420)),
        ('KNN', KNeighborsClassifier(n_neighbors=5))])

In [ ]:
voting_clf.fit(X_train, y_train)

Here's a handy way to see how each internal model does

In [ ]:
for name, clf in voting_clf.named_estimators_.items():
    print(f"{name} = {clf.score(X_test, y_test)}")

And the final accuracy. In this case we see a nice improvement

In [ ]:
print(f"Voting score = {voting_clf.score(X_test, y_test)}")

We plot the decision boundary. Notice the class-1 boundary is pretty complicated. This can be good if we're capturing real complexity in the data. Less good if the model is being influenced unduly by a noisy training set. In general, voting methods work to reduce the variance the the resulting model, which means your 'test accuracy' score is more reliable than with a single model alone. And hopefully better, too

In [ ]:
# Function to plot the decision boundary
def plot_decision_boundary(clf, X, y, ax=None, cmap='coolwarm', title="Decision Boundary"):
    if ax is None:
        ax = plt.gca()
    
    # Create a mesh grid
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 300),
                         np.linspace(y_min, y_max, 300))
    
    # Predict on the mesh grid
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Plot the decision boundary
    ax.contourf(xx, yy, Z, alpha=0.3, cmap=cmap)
    ax.scatter(X[y == 0, 0], X[y == 0, 1], color='blue', label='Class 0', edgecolor='k')
    ax.scatter(X[y == 1, 0], X[y == 1, 1], color='red', label='Class 1', edgecolor='k')
    ax.set_title(title)
    ax.legend()

# Plot the dataset and decision boundary
plt.figure(figsize=(8, 6))
plot_decision_boundary(voting_clf, X, y, title = "Hard Voting Classifier")
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

We can implement soft voting. The internal models each support a confidence score, and the Voting Classifier will use it.

In [ ]:
soft_voting_clf = VotingClassifier(
    estimators=[
        ('logistic regression', LogisticRegression(random_state = 420)),
        ('Decision Tree', DecisionTreeClassifier(random_state = 420)),
        ('KNN', KNeighborsClassifier())],
    voting = 'soft')

In [ ]:
soft_voting_clf.fit(X_train, y_train)
print(f"Soft voting accuracy score = {soft_voting_clf.score(X_test, y_test)}")

In [ ]:
# Plot the dataset and decision boundary
plt.figure(figsize=(8, 6))
plot_decision_boundary(soft_voting_clf, X, y, title = "Soft Voting Classifier")
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

## To - Do

Implement a voting classifier for the 'iris' dataset. Code to load the dataset is provided. Report the accuracy of your model. Use cross-validation for higher confidence in your result! Your voter can combine whicher classifiers you want.

In [ ]:
from sklearn.datasets import load_iris

# Load the Iris dataset
iris = load_iris()

# Extract features (X) and target labels (y)
X = iris.data    # Feature matrix (shape: 150 x 4)
y = iris.target  # Target vector (shape: 150,)

# Optionally, access feature and target names
feature_names = iris.feature_names
target_names = iris.target_names

print("Feature names:", feature_names)
print("Target names:", target_names)
print("Data shape:", X.shape)
print("Target shape:", y.shape)


In [ ]:
## Your Code Here!

## Bagging

Bagging is a tecnique that also uses multiple models. But in this case usually the same model is trained on different random subsets of the data. When these subsets are chosen with replacement, the result is a 'bagging' model. (Without replacement results in a 'pasting' model which usually has lower accuracy, but it never hurts to check.) We only look at 'bagging' here

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

### Decision Tree

First we try a single decision tree on the moons dataset

In [ ]:
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train,y_train)
print("Accuracy of Decision Tree: ", tree_clf.score(X_test, y_test))

Now we bag 100 trees, each one using a random sample of size 100 from the training set.

In [ ]:
bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators = 100,
                            max_samples = 100, random_state=42)

In [ ]:
bag_clf.fit(X_train, y_train)
print("Accuracy of Bagging 500 Trees: ", bag_clf.score(X_test, y_test))

Let's look at the decision boundary

In [ ]:
# Plot the dataset and decision boundary
plt.figure(figsize=(8, 6))
plot_decision_boundary(bag_clf, X_train, y_train)
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.show()

It's smoother than voting, and has a high accuracy. Bagging also tends to reduce variance when compared to a single model alone

### Random Forests

A random forest is a bagging model applied to decision trees. This is so common it exsits as a special classifier type

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes = 16, random_state=42)
rnd_clf.fit(X_train, y_train)

In [ ]:
rnd_clf.fit(X_train, y_train)
print("Accuracy of Random Forest: ", bag_clf.score(X_test, y_test))

### Feature Importance

It is possible by using these techniques to estimate a feature's importance. This estimate looks at the average amount of information gain in each feature, over several random decision trees. Features with the highest average information gain have the highest importance. We do a thorough treatment of feature importance in the titanic dataset.

This code includes some tricks/tips you maybe haven't seen before so read it carefully! You will need to modify it later.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the Titanic dataset
data = sns.load_dataset('titanic')

# Drop rows with missing target values
data = data.dropna(subset=['survived'])

# Define features and target
X_titanic = data[['pclass', 'sex', 'age', 'fare', 'embarked', 'sibsp', 'parch']]
y_titanic = data['survived']

# Handle missing values (impute with median for numerical, most frequent for categorical)
X_titanic.loc[:,'age'] = X_titanic['age'].fillna(X_titanic['age'].median())
X_titanic.loc[:,'embarked'] = X_titanic['embarked'].fillna(X_titanic['embarked'].mode()[0])

# Preprocess features
numerical_features = ['age', 'fare', 'sibsp', 'parch']
categorical_features = ['pclass', 'sex', 'embarked']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

# Create a pipeline with RandomForestClassifier
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_titanic, y_titanic, test_size=0.2, random_state=42)

# Fit the model
pipeline.fit(X_train, y_train)

# Extract the trained Random Forest model
rf = pipeline.named_steps['classifier']

# Get feature names in the order they appear in the preprocessed dataset
feature_names = numerical_features + list(
    pipeline.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(categorical_features)
)

# Get importances from the fitted moddel
importances = rf.feature_importances_

# Sort feature importances
# Zip feature names with their importances and sort them
feature_importances = sorted(
    zip(feature_names, importances), 
    key=lambda x: x[1], # this sorts by "y" in each (x,y) pair
    reverse=True
)

# Separate the sorted names and importance values
sorted_features, sorted_importances = zip(*feature_importances)

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.bar(range(len(sorted_importances)), sorted_importances, align='center')
plt.xticks(range(len(sorted_features)), sorted_features, rotation=90)
plt.title("Feature Importances from Titanic Dataset")
plt.xlabel("Feature")
plt.ylabel("Importance")
plt.tight_layout()
plt.show()


## To-Do

Determine the feature importance for the iris dataset. Make a similar plot to the one above (this dataset will not require all the fancy preprocessing.) We give code to load the dataset from sklearn

In [ ]:
from sklearn.datasets import load_iris

# Load the Iris dataset
iris = load_iris()

# Extract features (X) and target labels (y)
X = iris.data    # Feature matrix (shape: 150 x 4)
y = iris.target  # Target vector (shape: 150,)

# Optionally, access feature and target names
feature_names = iris.feature_names
target_names = iris.target_names

print("Feature names:", feature_names)
print("Target names:", target_names)
print("Data shape:", X.shape)
print("Target shape:", y.shape)


In [ ]:
## Your code Here

## Boosting

Boosting is a serial technique for improving training accuracy which works by training one model repeatedly, each time focusing on the wrong predictions. The same data is re-fed into the same model, but with a weight function applied to the outcomes where misclassified observations are given higher weight in a special loss function. Repeated iterations of boosting aim to reduce this loss function by improving the underlying model. AdaBoot and Gradient Boost are common examples here. XGBoost is the famous gradient boost algorithm.

### Ada Boost

Ada boost works by defining a weighted error function and increasing the weights of the missed observations. It was one of the first successful boosting techniques and is applicable to any algorithm. The learning rate controls how big the correction made is at each step. Smaller rates converge more slowly. Faster rates converge faster but are prone to over-stepping.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

We use more moons data

In [ ]:
X,y = make_moons(n_samples=5000, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [ ]:
ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=30, learning_rate = 0.5, random_state = 42)
ada_clf.fit(X_train, y_train)

In [ ]:
print("Ada Boost accuracy: ", ada_clf.score(X_test, y_test))

### To-Do

**TO-DO**: Make a plot of the ada-boost accuracy as a function of the learning rate. Use cross-validation on each datapoint in your graph.

In [ ]:
## Your code

### Gradient Boost

Gradient boost is similar to Ada Boost but focuses on minimizing the residual errors by fitting the error at each step, as opposed to Ada boost which simply tries to improve accuracy and change weights on the inputs.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
X,y = make_moons(n_samples=5000, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)
gb_clf = GradientBoostingClassifier(random_state = 42)
gb_clf.fit(X_train, y_train)

In [ ]:
print("Gradient Boost accuracy: ", gb_clf.score(X_test, y_test))

### To-Do

**To_Do**. Make a simple Decision Tree classifier and a Gradient Boosting Classifier for titanic. Use the `X_titanic` and `y_titanic` already loaded, and reuse the `pipeline` from the Titanic code, only changing what you need for your different models. Compare the resulting accuracies of the two models. Make sure you use cross validation. Reprt your best accuracy to the class!

In [ ]:
## your code

## Stacking

Stacking combines multiple models and makes their outputs to be input to a new model. This new model predicts the target based on the outputs of the input models.

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier

In this example, 3 models are input to a 3-layer neural network. You specify each base model and the final model in the classifier description. Notice this model always use cross validation internally to train the final_estimator. We pass in 5 as the number of cv folds to make in each training step.

In [ ]:
X,y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)
stacking_clf = StackingClassifier(
    estimators = [
        ('logistic', LogisticRegression(random_state = 42)),
        ('random forest', RandomForestClassifier(random_state=42)),
        ('svc', SVC(probability=True, random_state=42))
    ],
    final_estimator= MLPClassifier(random_state=42, hidden_layer_sizes=(40,10,5), learning_rate='adaptive', max_iter=1000),
    cv = 5
)

In [ ]:
stacking_clf.fit(X_train, y_train)

In [ ]:
print("Stacked accuracy: ", stacking_clf.score(X_test, y_test))

In [ ]:
print("Individual Accuracies\n")
for name, clf in stacking_clf.named_estimators_.items():
    print(f"{name} = {clf.score(X_test, y_test)}")

### To-Do

Apply stacking to 3 models on the Iris dataset. Print the individual and final accuracies.

In [ ]:
## Code Here

# Bigger To-Do

Load the smaller MNIST dataset we have used before (the full 60,000 dataset may take too long? I haven't tried). Make sure "X" is a list of 1D vectors. Using the 'feature importance' capabilities described above, determine which features (i.e. pixels) are most important. Make a 2D plot where the color of each pixel is a function of its importance. This will be a sort of heat-map of pixel importance in classifying digits.

In [ ]:
## code here